In [1]:
from helper import *
import numpy as np
import copy
import time

# Part 1

In [2]:
# Data structure
class Deck:
    def __init__(self, label, next_n = None):
#         self.pos = pos
        self.label = label
        self.next_n = next_n
    def __str__(self):
        if self.next_n is None:
            return '%s\t%s' %(self.label, 
                                      None)
        else:
            return '%s\t%s' %(self.label, self.next_n.label)

In [3]:
# Import data
test_data = read_file('Day22.txt')

In [4]:
data = test_data
pl1 = []
pl2 = []
cur_first = True
for row in test_data[1:]:
    if row[:6] == 'Player':
        cur_first = False
    elif row == '':
        pass
    else:
        if cur_first:
            pl1.append(int(row))
        else:
            pl2.append(int(row))

In [5]:
deck1_lst = pl1
deck2_lst = pl2

In [6]:
def build_deck(deck_lst):
    deck = Deck(deck_lst[0])

    prev_node = deck
    for val in deck_lst[1:]:
        node = Deck(val)
        prev_node.next_n = node
        prev_node = node
    return deck, node

In [7]:
def print_deck(deck):
    cur_node = deck
    while True:
        print(cur_node)
        cur_node = cur_node.next_n

        if cur_node is None:
            break

In [8]:
def get_deck(deck):
    cur_node = copy.deepcopy(deck)
    deck_lst = []
    while cur_node is not None:
        deck_lst.append(cur_node.label)
        cur_node = cur_node.next_n
    return deck_lst

In [9]:
# build decks
deck1, tail1 = build_deck(deck1_lst)
deck2, tail2 = build_deck(deck2_lst)

In [10]:
def play_round(deck1, deck2):
    p1, p2 = deck1, deck2
    deck1, deck2 = deck1.next_n, deck2.next_n
    if p1.label > p2.label:
#         print('p1 wins round')
        tail1 = deck1
        if tail1 is None:
            deck1 = p1
            tail1 = deck1
        while tail1.next_n is not None:
            tail1 = tail1.next_n
        tail1.next_n = p1
        p1.next_n = p2
        p2.next_n = None
        tail1 = p2
    else:
#         print('p2 wins round')
        tail2 = deck2
        if tail2 is None:
            deck2 = p2
            tail2 = deck2
        while tail2.next_n is not None:
            tail2 = tail2.next_n
        tail2.next_n = p2
        p2.next_n = p1
        p1.next_n = None
        tail2 = p1
    
    return deck1, deck2

In [11]:
pl_deck1 = copy.deepcopy(deck1)
pl_deck2 = copy.deepcopy(deck2)
pl_tail1 = copy.deepcopy(tail1)
pl_tail2 = copy.deepcopy(tail2)

rr = 1
while True:
    pl_deck1, pl_deck2 = \
        play_round(pl_deck1, pl_deck2)
    if pl_deck1 is None:
        print('Player 2 wins')
        break
    elif pl_deck2 is None:
        print('Player 1 wins')
        break
    rr += 1

print(rr)

Player 1 wins
981


In [12]:
rev_win = np.flip(get_deck(pl_deck1))
score = 0
for ii in range(1, len(rev_win)+1):
    score += ii*rev_win[ii-1]
print('Part one solution: %s' %score)

Part one solution: 32472


# Part 2

In [13]:
def key_deck(deck):
    return str(get_deck(deck))
    node = copy.deepcopy(deck)
    if node is None:
        return ''
    key = str(node.label)
    node = node.next_n
    while node is not None:
        key = key + ',' + str(node.label)
        node = node.next_n
    return key

In [14]:
def update(win_deck, lose_deck, p_win, p_lose):
    tail_win = win_deck
    if tail_win is None:
        win_deck = p_win
        tail_win = win_deck
    while tail_win.next_n is not None:
        tail_win = tail_win.next_n
    tail_win.next_n = p_win
#         print(tail1)
    p_win.next_n = p_lose
    p_lose.next_n = None
    tail_win = p_lose
    return win_deck

In [15]:
def deck_len(deck):
    count = 0
    cur_node = copy.deepcopy(deck)
    while cur_node is not None:
        count += 1
        cur_node = cur_node.next_n
    return count

In [16]:
def curtail_deck(deck, curt):
    track = 1
    cur_deck = copy.deepcopy(deck)
    cur_node = cur_deck
    for track in range(1, curt):
        cur_node = cur_node.next_n
    cur_node.next_n = None
    return cur_deck

In [17]:
def play_round_pt2(deck1, deck2, game_history, r_lev, all_hist):
    key_1 = key_deck(deck1)
    key_2 = key_deck(deck2)
    
    p1, p2 = deck1, deck2
    deck1, deck2 = deck1.next_n, deck2.next_n
    
    if key_1 in game_history['p1'] and \
            key_2 in game_history['p2']:
        deck1 = update(deck1, deck2, p1, p2)
        print('Win on history')
    
    elif deck_len(deck1) >= p1.label and deck_len(deck2) >= p2.label:
        # Play recurisve game
        print('Recursing to level %s' %(r_lev+1))
        
        rec_hist = {'p1': set([]), 'p2': set([])}
        rec_deck1 = curtail_deck(deck1, p1.label) 
        rec_deck2 = curtail_deck(deck2, p2.label)
        
        rec_key_1 = key_deck(rec_deck1)
        rec_key_2 = key_deck(rec_deck2)
        both_key = (rec_key_1, rec_key_2)
        
        rec_deck1_list = get_deck(rec_deck1)
        rec_deck1_len = len(rec_deck1_list)
        rec_deck1_max = np.max(rec_deck1_list)
        rec_deck1_min = np.min(rec_deck1_list)
        rec_deck2_list = get_deck(rec_deck2)
        rec_deck2_len = len(rec_deck2_list)
        rec_deck2_max = np.max(rec_deck2_list)
        rec_deck2_min = np.min(rec_deck2_list)
        rec_deck_min = min(rec_deck1_min, rec_deck2_min)
        
        if (rec_deck1_max > rec_deck2_max) and (rec_deck1_max \
                > (rec_deck1_len + rec_deck2_len - 2 - rec_deck_min)):
            # Know that deck 1 will win
            print('Aborting recursion, p1 will win')
            deck1 = update(deck1, deck2, p1, p2)
        
        elif both_key in all_hist.keys():
            print('Aborting recursion, know outcome')
            winner = all_hist[both_key]
            if winner == 'p1':
                deck1 = update(deck1, deck2, p1, p2)
            elif winner == 'p2':
                deck2 = update(deck2, deck1, p2, p1)
            else:
                raise ValueError
        else:
        
            while True:
                rec_deck1, rec_deck2, rec_hist = \
                    play_round_pt2(rec_deck1, rec_deck2, rec_hist, r_lev+1, all_hist)
                if rec_deck1 is None:
                    print('Player 2 wins, returning to level %s' %r_lev)
                    deck2 = update(deck2, deck1, p2, p1)
                    all_hist[both_key] = 'p2'
                    break
                elif rec_deck2 is None:
                    print('Player 1 wins, returning to level %s' %r_lev)
                    deck1 = update(deck1, deck2, p1, p2)
                    all_hist[both_key] = 'p1'
                    break

    elif p1.label > p2.label:
        deck1 = update(deck1, deck2, p1, p2)

    else:
        deck2 = update(deck2, deck1, p2, p1)

    if key_1 not in game_history['p1']:
        game_history['p1'].add(key_1)
    if key_2 not in game_history['p2']:
        game_history['p2'].add(key_2)
    
    return deck1, deck2, game_history

In [18]:
# NB: very slow

In [19]:
pl_deck1 = copy.deepcopy(deck1)
pl_deck2 = copy.deepcopy(deck2)
pl_tail1 = copy.deepcopy(tail1)
pl_tail2 = copy.deepcopy(tail2)

game_hist = {'p1': set([]), 'p2': set([])}
all_hist = {}

rr = 1
while True:
#     if rr % 10 == 0:
    print(rr)
    pl_deck1, pl_deck2, game_hist = \
        play_round_pt2(pl_deck1, pl_deck2, game_hist, 1, all_hist)
    if pl_deck1 is None:
        print('Player 2 wins')
        break
    elif pl_deck2 is None:
        print('Player 1 wins')
        break
    rr += 1

print(rr)

1
2
3
Recursing to level 2
Player 2 wins, returning to level 1
4
5
Recursing to level 2
Aborting recursion, p1 will win
6
7
8
9
10
11
12
Recursing to level 2
Recursing to level 3
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Player 2 wins, returning to level 2
Win on history
Win on history
Win on history
Recursing to level 3
Aborting recursion, p1 will win
Recursing to level 3
Aborting recursion, p1 will win
Win on history
Win on history
Win on history
Player 1 wins, returning to level 1
13
14
15
16
Recursing to level 2
Aborting recursion, p1 will win
17
18
Recursing to level 2
Aborting recursion, p1 will win
19
20
21
Recursing to level 2
Aborting recursion, p1 will win
22
23
24
25
26
27
Recursing to level 2
Aborting recursion, p1 will win
28
29
Recursing to level 2
Aborting recursion, p1 will win
30
31
Recursing to level 2
Aborting recursion, p1 will win
32
33
34
35
Recursing to level 2
Player 2 wins, returning to 

Recursing to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Aborting recursion, p1 will win
Recursing to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Aborting recursion, know outcome
Player 2 wins, returning to level 1
176
177
Recursing to level 2
Aborting recursion, p1 will win
178
179
180
181
Recursing to level 2
Aborting recursion, p1 will win
182
183
184
185
Recursing to level 2
Aborting recursion, p1 will win
186
Recursing to level 2
Aborting recursion, p1 will win
187
188
189
190
Recursing to level 2
Aborting recursion, p1 will win
191
192
193
194
195
196
197
Recursing to level 2
Aborting recursion, p1 will win
198
199
Recursing to level 2
Player 2 wins, returning to level 1
200
201
202
203
204
205
206
207
208
209
210
211
Recursing to level 2
Player 2 wins, returning to level 1
212
213
214
215
Recursing to level 2
Recursing to level 3
Player 2 wins, returning to level 2
Recursing to lev

Win on history
Win on history
Recursing to level 4
Win on history
Win on history
Player 2 wins, returning to level 3
Win on history
Win on history
Recursing to level 4
Player 2 wins, returning to level 3
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Recursing to level 4
Win on history
Win on history
Win on history
Recursing to level 5
Player 2 wins, returning to level 4
Recursing to level 5
Win on history
Win on history
Player 1 wins, returning to level 4
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 3
Win on history
Win on history
Recursing to level 4
Win on history
Win on history
Player 1 wins, returning to level 3
Recursing to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Aborting recursion, p1 will win
Win on history
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Player 2 wins, returning to l

Win on history
Win on history
Win on history
Win on history
Recursing to level 4
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 3
Win on history
Win on history
Recursing to level 4
Aborting recursion, p1 will win
Player 1 wins, returning to level 2
Recursing to level 3
Aborting recursion, p1 will win
Recursing to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Win on history
Win on history
Win on history
Recursing to level 5
Aborting recursion, p1 will win
Recursing to level 5
Aborting recursion, p1 will win
Win on history
Win on history
Win on history
Win on history
Recursing to level 5
Player 2 wins, returning to level 4
Win on history
Win on hist

Win on history
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Win on history
Win on history
Win on history
Win on history
Recursing to level 4
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 3
Recursing to level 4
Aborting recursion, p1 will win
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Aborting recursion, p1 will win
Recursing to level 3
Recursing to level 4
Win on history
Win on history
Player 1 wins, returning to level 3
Recursing to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Player 2 wins, returning to

...

Recursing to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Aborting recursion, p1 will win
Recursing to level 4
Recursing to level 5
Win on history
Recursing to level 6
Aborting recursion, p1 will win
Win on history
Win on history
Player 1 wins, returning to level 4
Win on history
Win on history
Win on history
Recursing to level 5
Aborting recursion, p1 will win
Recursing to level 5
Aborting recursion, p1 will win
Win on history
Win on history
Win on history
Player 1 wins, returning to level 3
Recursing to level 4
Aborting recursion, p1 will win
Recursing to level 4
Recursing to level 5
Win on history
Win on history
Win on history
Player 1 wins, returning to level 4
Recursing to level 5
Win on history
Player 1 wins, returning to level 4
Recursing to level 5
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 4
Rec

Win on history
Win on history
Recursing to level 6
Aborting recursion, p1 will win
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 4
Recursing to level 5
Recursing to level 6
Aborting recursion, p1 will win
Recursing to level 6
Win on history
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 5
Recursing to level 6
Player 2 wins, returning to level 5
Recursing to level 6
Player 2 wins, returning to level 5
Recursing to level 6
Player 2 wins, returning to level 5
Recursing to level 6
Player 2 wins, returning to level 5
Player 2 wins, returning to level 4
Recursing to level 5
Aborting recursion, p1 will win
Recursing to level 5
Recursing to level 6
Player 2 wins, returning to level 5
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning 

Win on history
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Recursing to level 4
Recursing to level 5
Win on history
Win on history
Win on history
Player 1 wins, returning to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Win on history
Win on history
Win on history
Recursing to level 5
Aborting recursion, p1 will win
Win on history
Win on history
Player 1 wins, returning to level 3
Recursing to level 4
Aborting recursion, p1 will win
Recursing to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Recursing to level 5
Win on history
Win on history
Win on history
Player 1 wins, returning to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Recursing to

Win on history
Win on history
Player 1 wins, returning to level 2
Recursing to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Recursing to level 5
Win on history
Player 1 wins, returning to level 4
Win on history
Win on history
Recursing to level 5
Aborting recursion, p1 will win
Win on history
Recursing to level 5
Aborting recursion, p1 will win
Win on history
Player 1 wins, returning to level 3
Recursing to level 4
Aborting recursion, know outcome
Recursing to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Recursing to level 5
Player 2 wins, returning to level 4
Recursing to level 5
Win on history
Win on history
Player 1 wins, returning to level 4
Recursing to level 5
Aborting recursion, p1 will win
Recursing to level 5
Player 2 wins, returning to level 4
Player 2 wins, returning to level 3
Recursing to level 4
Aborting recursion, p1 will win
Recursing to level 4
Aborting recursion, p1 will win
Recursing to level 4
Recursing to

Win on history
Win on history
Win on history
Win on history
Win on history
Recursing to level 3
Aborting recursion, p1 will win
Win on history
Win on history
Win on history
Win on history
Player 1 wins, returning to level 1
584
585
Recursing to level 2
Player 2 wins, returning to level 1
586
587
Recursing to level 2
Player 2 wins, returning to level 1
588
589
590
591
Recursing to level 2
Aborting recursion, p1 will win
592
593
594
595
596
597
598
599
600
601
602
603
Recursing to level 2
Aborting recursion, p1 will win
604
605
Recursing to level 2
Aborting recursion, p1 will win
606
607
Recursing to level 2
Aborting recursion, p1 will win
608
609
610
611
612
613
614
615
Recursing to level 2
Recursing to level 3
Recursing to level 4
Player 2 wins, returning to level 3
Player 2 wins, returning to level 2
Recursing to level 3
Player 2 wins, returning to level 2
Player 2 wins, returning to level 1
616
617
618
619
Recursing to level 2
Aborting recursion, p1 will win
620
Recursing to level 2


In [20]:
rev_win = np.flip(get_deck(pl_deck1))
rev_win
score = 0
for ii in range(1, len(rev_win)+1):
    score += ii*rev_win[ii-1]
print('Part two solution: %s' %score)

Part two solution: 36463
